In [1]:
import altair as alt
import seaborn as sns
from altair import datum
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

regioni_df = get_processed_dataset('dpc_regioni')
today = regioni_df.groupby('regione').last().reset_index()

## Riepilogo per regione

### Casi per regione

In [2]:
from altair import datum

c = alt.Chart(regioni_df).mark_line().encode(
        x='data:T',
        color='regione',
        tooltip=['regione', 'data', 'totale_casi', 'deceduti', 'totale_casi_per_1M_pop', 'deceduti_per_1M_pop']
    ).properties(
        height=600,
        width=800
    ).transform_filter(
        (datum.totale_casi > 1000)
    ).interactive()
    
@interact(campo=['totale_casi', 'deceduti'], logarithm=True, per1Mpop=False)
def plot_province(campo, logarithm, per1Mpop):    
    y = campo
    if per1Mpop:
        y += '_per_1M_pop'
    if logarithm:
        return c.encode(y = alt.Y(f'{y}:Q', scale=alt.Scale(type='log')))
    else:
        return c.encode(y = alt.Y(f'{y}:Q'))

interactive(children=(Dropdown(description='campo', options=('totale_casi', 'deceduti'), value='totale_casi'),…

In [3]:
@interact(campo=['totale_casi', 'deceduti', 'tamponi'], normalize=True)
def plot(campo, normalize):
    stack = "normalize" if normalize else "zero"
    
    return alt.Chart(regioni_df).mark_area().encode(
        alt.Y(campo, stack=stack),#, scale=alt.Scale(type='log')),
        x='data:T',
        color=alt.Color('regione', scale=alt.Scale(scheme='category20')),
        tooltip=['regione', 'data', 'totale_casi', 'deceduti', 'tamponi']
    ).properties(
        height=600,
        width=800
    ).transform_filter(
        (datum.totale_casi > 0)
    ).interactive()

interactive(children=(Dropdown(description='campo', options=('totale_casi', 'deceduti', 'tamponi'), value='tot…

In [4]:
@interact(campo=['percentuale_positivi', 'percentuale_nuovi_positivi', 'incremento', 'nuovi_tamponi', 'nuovi_positivi', 'nuovi_deceduti', 'tamponi', 'terapia_intensiva', 'totale_positivi', 'letalita', 'totale_casi_per_1M_pop', 'deceduti_per_1M_pop'], giorni_indietro=(-10,0))
def plot(campo='nuovi_deceduti', giorni_indietro=0):
    df = regioni_df.groupby('regione').nth(giorni_indietro-1).reset_index()
    c = alt.Chart(df).mark_bar().encode(
        x=campo,
        y=alt.Y('regione:N', sort='-x'),
    ).properties(
        height=600,
        width=800
    ).transform_filter(
            (datum.percentuale_positivi > 0)
    ).properties(title=str(df.data.iloc[0])).interactive()
    return c

interactive(children=(Dropdown(description='campo', index=5, options=('percentuale_positivi', 'percentuale_nuo…

In [5]:
regioni_df[regioni_df.codice_regione==22]

,data,stato,codice_regione,regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,nuovi_tamponi,nuovi_casi_testati,incremento,percentuale_positivi,percentuale_nuovi_positivi,letalita,totale_casi_per_1M_pop,deceduti_per_1M_pop,nuovi_deceduti_per_1M_pop,nuovi_positivi_per_1M_pop
1394,2020-02-24 18:00:00,ITA,22,P.A. Trento,46.068935,11.121231,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
1395,2020-02-25 18:00:00,ITA,22,P.A. Trento,46.068935,11.121231,0,0,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1396,2020-02-26 18:00:00,ITA,22,P.A. Trento,46.068935,11.121231,0,0,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1397,2020-02-27 18:00:00,ITA,22,P.A. Trento,46.068935,11.121231,0,0,0,0,...,29.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1398,2020-02-28 18:00:00,ITA,22,P.A. Trento,46.068935,11.121231,0,0,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,2020-05-11 17:00:00,ITA,22,P.A. Trento,46.068935,11.121231,84,11,95,640,...,1359.0,1162.0,0.046544,13.440726,0.172117,10.309518,7941.260178,818.705669,3.696188,3.696188
1472,2020-05-12 17:00:00,ITA,22,P.A. Trento,46.068935,11.121231,80,9,89,578,...,1936.0,1112.0,0.139438,13.007073,0.539568,10.341622,7952.348743,822.401857,3.696188,11.088564
1473,2020-05-13 17:00:00,ITA,22,P.A. Trento,46.068935,11.121231,76,9,85,488,...,2663.0,1580.0,0.208720,12.440136,0.569620,10.343228,7968.981589,824.249951,1.848094,16.632847
1474,2020-05-14 17:00:00,ITA,22,P.A. Trento,46.068935,11.121231,66,9,75,430,...,2846.0,1697.0,0.069525,11.867763,0.176783,10.382387,7974.525871,827.946139,3.696188,5.544282
